## **First project for Cryptography**

We need to implement several function:
- CTR and CBC modes for AES-256
- Encryption and decryption modes
- Check if the decrypted message is identical to the original plaintext

In [3]:
"""
For this project, we need to import AES, get_random_bytes and Counter from the Pycryptodome library.
Then for the encoding, we need the b64encode, b64decode module
Finally, to calculate the time for encryption and decryption, we need the time module and to have the location of the current working directory, we need os module.
"""
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import Crypto.Util.Counter
from base64 import b64encode, b64decode
from time import time
import os

"""
To implement our encryption algorithm, we need to specify a key, an iv and the block size.
We create a variable ctr to set our own counter for using the right iv for the ctr library encryption
You can see that each variable starting with cipher corresponds to a specific encryption algorithm.
Finally, it is mandatory to specify an encryption algorithm with a different variable to allow encryption and decryption within the same file in the case of libraries.
"""
key = get_random_bytes(32)
iv = get_random_bytes(16)
block_size = AES.block_size
ctr = Crypto.Util.Counter.new(128, initial_value=int.from_bytes(iv, "big"))
cipher_ecb = AES.new(key, AES.MODE_ECB)
cipher_cbc = AES.new(key, AES.MODE_CBC, iv)
cipher_cbc_dec = AES.new(key, AES.MODE_CBC, iv)
cipher_ctr = AES.new(key, AES.MODE_CTR, counter=ctr)
cipher_ctr_dec = AES.new(key, AES.MODE_CTR, counter=ctr)


"""
In this section, we will read the original file in bytes format directly through "rb".
Then I will add a manual padding if the text is not a multiple of the block size.
The main method for padding is PKCS#7 but here I input the padding myself with a bit of 0.
And finally I will transform the whole text padding into a portion of the same length as the block.
"""
def read_and_convert_plaintext_file(location):
    with open(location + "/plaintext.txt", "rb") as f:
        plaintext = f.read()
        plaintext_with_pad = plaintext + b"\0" * (block_size - len(plaintext) % block_size)
        plaintext_block = [plaintext_with_pad[i:i + block_size] for i in range(0, len(plaintext_with_pad), block_size)]
        f.close()

    return plaintext_block

"""
In this section, we will read the ciphertext file in bytes format directly through "rb".
Then I will decode this file with b64decode.
And finally I will transform the whole ciphertext into a portion of the same length as the block.
"""
def read_and_convert_ciphertext_file(location):
    with open(location + "/ciphertext.txt", "rb")  as f:
        cipher_text = b64decode(f.read())
        cipher_block = [cipher_text[i:i + block_size] for i in range(0, len(cipher_text), block_size)]
        f.close()

    return cipher_block

"""
In this section, we will create the ciphertext file in bytes format directly through "wb".
Then I will encode this file with b64encode.
"""
def create_ciphertext_file(cipher_text, location):
    with open(location + "/ciphertext.txt", "wb") as f:
        f.write(b64encode(cipher_text))
        f.close()

"""
In this section, we will write the decrypted plaintext file directly in string format through "w".
Then I will decode this file with b64decode.
And finally I will transform the whole ciphertext into a portion of the same length as the block.
"""
def create_plaintext_file(plaintext, location):
    with open(location + "/decrypted_plaintext.txt", "w") as f:
        f.write(plaintext)
        f.close()

"""
In this function, we return the operation xor from bit to bit of the elements passed in parameters.
The zip method allows to combine the two parameters and return a tuple.
"""
def xor_operation(string1, string2):
    return bytes(element1 ^ element2 for element1, element2 in zip(string1, string2))

"""
In this function, I will manually decrypt the plaintext from the AES_ECB encryption algorithm to AES_CBC.
First, I will XOR the first block of plaintext with the IV and then encrypt it.
For all other iterations, I will XOR the plaintext block with the previously created ciphertext block and then encrypt it.
Finally, I'll concatenate everything in the cipher_text variable.
"""
def aes_cbc_encryption(plaintext_block):
    cipher_text = bytes()
    cipher_block = bytes()
    for i in range(len(plaintext_block)):
        if i == 0:
            try:
                xor = xor_operation(plaintext_block[i], iv)
                cipher_block = cipher_ecb.encrypt(xor)
            except Exception as error:
                print(error)
        else:
            try:
                xor = xor_operation(plaintext_block[i], cipher_block)
                cipher_block = cipher_ecb.encrypt(xor)
            except Exception as error:
                print(error)

        cipher_text += cipher_block

    return cipher_text


"""
In this function, I will manually decrypt the plaintext from the manual AES_CBC encryption algorithm.
First, I will decrypt the ciphertext then XOR the first block of text obtained with the IV.
For all other iterations, I will decrypt the ciphertext and then XOR the text block obtained with the old ciphertext block.
Finally, I'll concatenate everything in the plaintext_encode_with_pad variable since there is still padding and encoding.
So I will remove the padding with rstrip and decode to utf-8.
"""
def aes_cbc_decryption(cipher_block):
    plaintext_encode_with_pad = bytes()
    plaintext_block = bytes()

    for i in range(len(cipher_block)):
        if i == 0:
            try:
                xor = cipher_ecb.decrypt(cipher_block[i])
                plaintext_block = xor_operation(xor, iv)
            except Exception as error:
                print(error)
        else:
            try:
                xor = cipher_ecb.decrypt(cipher_block[i])
                plaintext_block = xor_operation(xor, cipher_block[i - 1])
            except Exception as error:
                print(error)

        plaintext_encode_with_pad += plaintext_block

    plaintext = plaintext_encode_with_pad.rstrip(b"\0").decode("utf-8")

    return plaintext

"""
In this function, I will manually encrypt the plaintext using the AES_ECB encryption algorithm to AES_CTR.
First, I will encrypt the ciphertext by specifying the iv used and increment it by 1. To do this, we transform it into an integer to increment it and then transform it back into bytes.
Then we make an xor between the cipher text with the iv and the plaintext block.
We store the cipher text in the variable cipher_text and return it.
"""
def aes_ctr_encryption(plaintext_block):
    cipher_text = bytes()

    for i in range(len(plaintext_block)):
        cipher_with_iv = cipher_ecb.encrypt((int.from_bytes(iv, "big")+i).to_bytes(block_size, byteorder='big'))
        cipher_block = xor_operation(cipher_with_iv, plaintext_block[i])
        cipher_text += cipher_block

    return cipher_text


"""
In this function, I will manually decrypt the plaintext using the AES_ECB encryption algorithm.
First, I will encrypt the ciphertext by specifying the iv used and increment it by 1. To do this, we transform it into an integer to increment it and then transform it back into bytes.
Yes, we make the same operation than encryption because ctr is a XOR function.
So, we encrypt the encrypted ciphertext to retrieve the plaintext.
Then we make an xor between the cipher text with the iv and the cipher text block.
We store the plaintext in the variable plaintext.
Before returning the plaintext, we need to rstrip for the padding and decode the text in utf-8.
"""
def aes_ctr_decryption(cipher_block):
    plaintext = bytes()

    for i in range(len(cipher_block)):
        cipher_with_iv = cipher_ecb.encrypt((int.from_bytes(iv, "big") + i).to_bytes(block_size, byteorder="big"))
        cipher_block_2 = xor_operation(cipher_with_iv, cipher_block[i])
        plaintext += cipher_block_2

    return plaintext.rstrip(b"\0").decode("utf-8")

"""
In the next functions, we use the same principle but using the libraries provided by pycryptodome AES_MODE_CBC and AES_MODE_CTR to allow time comparison.
"""
def cbc_normal_encryption(location):
    start_time_library = time()
    print("BEGINS CBC LIBRARY ENCRYPTION")
    print("1. Read the plaintext file...")
    with open(location + "/plaintext.txt", "rb") as f:
        plaintext = f.read()
        plaintext_with_pad = plaintext + b"\0" * (block_size - len(plaintext) % block_size)
        f.close()

    print("2. Encrypt plaintext file...")
    cipher_text = cipher_cbc.encrypt(plaintext_with_pad)

    print("3. Create the cipher text file...")
    with open(location + "/ciphertext.txt", "wb") as f:
        f.write(b64encode(cipher_text))
        f.close()

    print("4. End of encryption in : {:.3f} seconds".format(time() - start_time_library))

def cbc_normal_decryption(location):
    start_time_library = time()
    print("BEGINS CBC LIBRARY DECRYPTION...")
    print("1. The decryption begin...")
    with open(location + "/ciphertext.txt", "rb") as f:
        cipher_text = b64decode(f.read())
        f.close()

    print("2. Decrypt ciphertext file...")
    plaintext_encode = cipher_cbc_dec.decrypt(cipher_text)
    plaintext = plaintext_encode.rstrip(b"\0").decode("utf-8")

    print("3. Create the decrypted plaintext file...")
    with open(location + "/decrypted_plaintext.txt", "w") as f:
        f.write(plaintext)
        f.close()

    print("4. End of decryption in : {:.3f} seconds".format(time() - start_time_library))

def ctr_normal_encryption(location):
    start_time_library = time()
    print("BEGINS CTR LIBRARY ENCRYPTION")
    print("1. Read the plaintext file...")
    with open(location + "/plaintext.txt", "rb") as f: # read the plaintext file and store it into a variable f
        plaintext = f.read()# transform string type into bytes type for padding, encryption and decryption
        plaintext_with_pad = plaintext + b"\0" * (block_size - len(plaintext) % block_size)  # add padding to have a multiple of 16 bytes (block size)
        f.close()

    print("2. Encrypt plaintext file...")
    cipher_text = cipher_ctr.encrypt(plaintext_with_pad)

    print("3. Create the cipher text file...")
    with open(location + "/ciphertext.txt", "wb") as f:
        f.write(b64encode(cipher_text))
        f.close()

    print("4. End of encryption in : {:.3f} seconds".format(time() - start_time_library))
def ctr_normal_decryption(location):
    start_time_library = time()
    print("BEGINS CTR LIBRARY DECRYPTION...")
    print("1. Read the plaintext file...")
    with open(location + "/ciphertext.txt", "rb") as f:
        cipher_text = b64decode(f.read())
        f.close()

    print("2. Decrypt ciphertext file...")
    plaintext_encode = cipher_ctr_dec.decrypt(cipher_text)
    plaintext = plaintext_encode.rstrip(b"\0").decode("utf-8")

    print("3. Create the decrypted plaintext file...")
    with open(location + "/decrypted_plaintext.txt", "w") as f:
        f.write(plaintext)
        f.close()

    print("4. End of decryption in : {:.3f} seconds".format(time() - start_time_library))

"""
The main function allows the user to choose his encryption algorithm from 1 to 4 and to call all the necessary functions according to the chosen algorithm.
"""
def main():
    print(
        "###################################################################\n"
        "# Project 1 for Cryptography:                                     #\n"
        "# Implement AES-CBC and AES-CTR encryption and decryption mode    #\n"
        "# By Edouard Bettignies Masi-M1                                   #\n"
        "###################################################################"
    )

    choose_cipher = int(input("Would you like to use the manuel CBC (1) or manuel CTR (2) or library CBC (3) or library CTR (4) : "))
    location = os.getcwd()

    try:
        plaintext_block = read_and_convert_plaintext_file(location)
    except FileNotFoundError as e:
        print("You enter a wrong path or the file does not exist!")

    start_time_encryption = time()


    if choose_cipher == 1:
        print("BEGINS ENCRYPTION")
        print("1. Encrypt file using AES-CBC-256...")
        cipher_text = aes_cbc_encryption(plaintext_block)
        print("2. Create ciphertext file with base 64 encode...")
        create_ciphertext_file(cipher_text, location)
        print("3. File created in {:.3f} seconds".format(time() - start_time_encryption))

        start_time_decryption = time()
        print("BEGINS DECRYPTION...")
        print("1. Read and convert ciphertext file...")
        cipher_block = read_and_convert_ciphertext_file(location)
        print("2. Decrypt ciphertext file...")
        plaintext = aes_cbc_decryption(cipher_block)
        print("3. Create plaintext file...")
        create_plaintext_file(plaintext, location)
        print("4. File created in {:.3f} seconds".format(time() - start_time_decryption))

    elif choose_cipher == 2:
        print("BEGINS ENCRYPTION")
        print("1. Encrypt file using AES-CTR-256...")
        cipher_text = aes_ctr_encryption(plaintext_block)
        print("2. Create ciphertext file with base 64 encode...")
        create_ciphertext_file(cipher_text, location)
        print("3. File created in {:.3f} seconds".format(time() - start_time_encryption))
        print("BEGINS DECRYPTION...")
        start_time_decryption = time()
        print("1. Read and convert ciphertext file...")
        cipher_block = read_and_convert_ciphertext_file(location)
        print("2. Decrypt ciphertext file...")
        plaintext = aes_ctr_decryption(cipher_block)
        print("3. Create plaintext file...")
        create_plaintext_file(plaintext, location)
        print("4. File created in {:.3f} seconds".format(time() - start_time_decryption))

    elif choose_cipher == 3:
        cbc_normal_encryption(location)
        cbc_normal_decryption(location)
    elif choose_cipher == 4:
        ctr_normal_encryption(location)
        ctr_normal_decryption(location)

if __name__ == "__main__":
    main()


###################################################################
# Project 1 for Cryptography:                                     #
# Implement AES-CBC and AES-CTR encryption and decryption mode    #
# By Edouard Bettignies Masi-M1                                   #
###################################################################
BEGINS ENCRYPTION
1. Encrypt file using AES-CBC-256...
2. Create ciphertext file with base 64 encode...
3. File created in 1.789 seconds
BEGINS DECRYPTION...
1. Read and convert ciphertext file...
2. Decrypt ciphertext file...
3. Create plaintext file...
4. File created in 1.409 seconds
